In [1]:
#READ_CSV, READ_TEXT, Import Pandas, os

import pandas as pd
import os

def read_csv(path):
    df = pd.read_csv(path, encoding='unicode_escape')
    df['PersonID'] = os.path.split(path)[1].split('_')[0][6:]
    df['ImageID'] = os.path.split(path)[1].split('.')[0][6:]
    return df

#只有 Text 跟 Caption 會用
def read_text(path, colName):
    df = pd.read_csv(path, sep='\t', encoding='unicode_escape', names=[colName])
    filename = os.path.split(path)[1]
    if 'text' in filename:
        df['TextID'] = filename[6:-4]
    else:
        df['ImageID'] = os.path.split(path)[1].split('.')[0][6:-7]
    return df

def load_FineData():
    return pd.read_csv('FineData.csv', encoding='unicode_escape')


In [2]:
#Reading all file

DataPath = 'MC2-Image-Data/'

#Full DataPaths
CsvList = []
#ImageList = [] 
CapList = [] 
TextList = []

#ImagesNames = []

for r, d, files in os.walk(DataPath):

    if 'TrainingImages' in r:
            continue

    for _file in files:

        if _file.endswith('.csv'):
            CsvList.append(r + '/' + _file)
        #elif _file.endswith('.jpg'):
        #    ImageList.append(r + '/' +_file)
        #    ImagesNames.append(_file)
        elif 'caption.txt' in _file:
            CapList.append(r + '/' +_file)
        elif 'text' in _file:
            TextList.append(r + '/' +_file)

In [3]:
# Make Caption/Text/Csv DataFrame

tmp = []
for path in CapList:
    tmp.append(read_text(path, 'Caption'))

cap_df = pd.concat(tmp, ignore_index=True)

tmp = []
for path in TextList:
    tmp.append(read_text(path, 'Text'))

text_df = pd.concat(tmp, ignore_index=True)

tmp = []
for path in CsvList:
    tmp.append(read_csv(path))

csv_df = pd.concat(tmp, ignore_index=True)

In [4]:
# Sort by Simply ImageID
csv_df['Sort'] = csv_df['ImageID'].apply(lambda x : int(x.replace('_', ""))) 
csv_df = csv_df.sort_values(by='Sort', ignore_index=True)
csv_df = csv_df.drop(columns = ['Sort'])
csv_df



,x,y,Width,Height,Score,Label,PersonID,ImageID
0,1479,11,1922,1583,0.33861,birdCall,1,1_1
1,2554,313,955,888,0.28139,eyeball,1,1_1
2,1855,1724,578,1079,0.34569,pinkCandle,1,1_1
3,14,666,1465,2309,0.42683,pumpkinNotes,1,1_1
4,1560,1301,1197,615,0.32564,blueSunglasses,1,1_1
...,...,...,...,...,...,...,...,...
4486,1055,795,1953,1602,0.30762,birdCall,40,40_10
4487,1884,1176,985,1291,0.30588,cloudSign,40,40_10
4488,847,1556,1590,1544,0.58092,redWhistle,40,40_10
4489,1279,957,1521,1371,0.48834,hairClip,40,40_10


In [5]:
# Save Files
csv_df.to_csv('FineData.csv', encoding='unicode_escape')
csv_df


,x,y,Width,Height,Score,Label,PersonID,ImageID
0,1479,11,1922,1583,0.33861,birdCall,1,1_1
1,2554,313,955,888,0.28139,eyeball,1,1_1
2,1855,1724,578,1079,0.34569,pinkCandle,1,1_1
3,14,666,1465,2309,0.42683,pumpkinNotes,1,1_1
4,1560,1301,1197,615,0.32564,blueSunglasses,1,1_1
...,...,...,...,...,...,...,...,...
4486,1055,795,1953,1602,0.30762,birdCall,40,40_10
4487,1884,1176,985,1291,0.30588,cloudSign,40,40_10
4488,847,1556,1590,1544,0.58092,redWhistle,40,40_10
4489,1279,957,1521,1371,0.48834,hairClip,40,40_10


In [7]:
# Load csv
Df = load_FineData()
Df

NameError: name 'load_FineData' is not defined

In [9]:
pivot = Df.pivot(index='ImageID', columns='Label', values='Score')
pivot

ValueError: Index contains duplicate entries, cannot reshape